In [ ]:

from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType, utility

In [ ]:
'''
collection 集合 = 一个表
field 字段 = 列名称
index 索引 = 索引名称
entity 实体 = 一条数据 = (index ,field1,field2)
'''

In [ ]:
# milvus配置
MILVUS_HOST = "localhost"
MILVUS_PORT = 19530
COLLECTION_NAME = "snomed"
DIM = 1024  # 向量维度
USING = "testing"

创建并连接数据表

In [ ]:
# 1 建立连接
# 对于没有指定using的操作会默认使用default连接
connections.connect(
        alias="default",
        host=MILVUS_HOST,
        port=MILVUS_PORT
    )

connections.connect(
        alias=USING, # 自定义 用于区分多个不同的连接
        host=MILVUS_HOST,
        port=MILVUS_PORT
    )


In [ ]:
# 删除旧表(在测试时使用)
if utility.has_collection(COLLECTION_NAME,using=USING):
    utility.drop_collection(COLLECTION_NAME,using=USING)
    print(f"🗑️  删除旧集合: {COLLECTION_NAME}")

In [ ]:
# 2 创建表 同时 获取collection对象

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True), # 主键
    
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=DIM), # 向量

    FieldSchema(name="concept_id", dtype=DataType.VARCHAR, max_length=50), # 
    FieldSchema(name="concept_name", dtype=DataType.VARCHAR, max_length=500), # 
    FieldSchema(name="domain_id", dtype=DataType.VARCHAR, max_length=20), # 
   
    FieldSchema(name="vocabulary_id", dtype=DataType.VARCHAR, max_length=20),
    FieldSchema(name="concept_class_id", dtype=DataType.VARCHAR, max_length=20),
    FieldSchema(name="standard_concept", dtype=DataType.VARCHAR, max_length=1),
    FieldSchema(name="concept_code", dtype=DataType.VARCHAR, max_length=50),

    FieldSchema(name="valid_start_date", dtype=DataType.VARCHAR, max_length=10),
    FieldSchema(name="valid_end_date", dtype=DataType.VARCHAR, max_length=10),
]

# 构建集合的schema
schema = CollectionSchema(
    fields=fields, # 列
    description="snomed 医疗概念", # 描述
    enable_dynamic_field=True, # 允许动态字段
)

# 获取collection对象
# 不提供schema就表示连接到已有表
collection = Collection(name=COLLECTION_NAME,schema=schema,using=USING)

# 在创建集合后添加索引 加速搜索
collection.create_index(
    field_name="vector", # 对哪列构建索引
    index_params= {
        "index_type": "HNSW",          # 性能优于 IVF_FLAT
        "metric_type": "COSINE",       # 余弦相似度
        # 不同 索引类型 index_type 需要不同的参数
        "params": {
            "M": 16,                    # 连接数
            "efConstruction": 256       # 构建索引时的搜索范围
        }
    }
)

加载embedding模型

In [ ]:
import torch
from sentence_transformers import SentenceTransformer

In [ ]:
em_model = SentenceTransformer(
    model_name_or_path='BAAI/bge-m3', 
    # model_name_or_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct',
    device='cuda:0' if torch.cuda.is_available() else 'cpu',
    trust_remote_code=True,

    model_kwargs={'dtype': torch.bfloat16}  # 使用 bfloat16
)

加载数据

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('SNOMED_ALL.csv')

将数据向量化后写入数据库

In [ ]:
# 用什么字段搜索 就把什么构建成向量
import tqdm

batch_size = 1024

for start_idx in tqdm.tqdm(range(0,len(df),batch_size)):

    # 获取一个批次的数据
    end_idx = min(start_idx + batch_size, len(df)) # 防止超出索引
    batch_df = df.iloc[start_idx:end_idx]

    # 对 concept_name 列数据进行量化
    # 把 1024 个数据 放入 列表
    str_list = batch_df['concept_name'].astype(str).tolist()

    # 向量化
    embeddings = em_model.encode(
        str_list, 
        batch_size=128,              # ⭐ GPU内部批处理大小
        show_progress_bar=False,
        convert_to_numpy=True,       # 转为numpy加速
        normalize_embeddings=False    # 如果不需要归一化
    )

    # 构造要写入的 1024 条数据
    data = []
    for i in range(len(embeddings)):
        row = batch_df.iloc[i]  # 获取 batch_df 的第 i 行
        item = {
            "vector": embeddings[i].tolist(),

            "concept_id": str(row['concept_id']),
            "concept_name": str(row['concept_name']),
            "domain_id": str(row['domain_id']),

            "vocabulary_id": str(row['vocabulary_id']),
            "concept_class_id": str(row['concept_class_id']),
             # 处理NaN，如果是NaN则使用空字符串，否则转为字符串
            "standard_concept": "" if pd.isna(row['standard_concept']) else str(row['standard_concept']),
            "concept_code": str(row['concept_code']),

            "valid_start_date": str(row['valid_start_date']),
            "valid_end_date": str(row['valid_end_date']),
        }
        data.append(item)
    
    # 批量写入数据
    collection.insert(data)

